Parsing responses using GPT-4o

Collecting the responses from all the models.

In [2]:
import pandas as pd

df_prompts = pd.read_csv('prompts.csv', sep=';', encoding='utf-8-sig')
df_openai = pd.read_csv('responses_openAI.csv', sep=';', encoding='utf-8-sig')
df_yandexgpt = pd.read_csv('responses_yandexGPT.csv', sep=';', encoding='utf-8-sig')

Merging responses into one dataframe.

In [3]:
df_gpt4o = df_openai[df_openai['model'] == 'gpt-4o'].reset_index()[['question','response']].rename(columns={'response':'gpt-4o'})
df_gpt35turbo = df_openai[df_openai['model'] == 'gpt-3.5-turbo'].reset_index()[['question','response']].rename(columns={'response':'gpt-3.5-turbo'})
df_openai_unstack = pd.merge(df_gpt4o,df_gpt35turbo,left_index=True,right_index=True)

df_responses = pd.merge(df_openai_unstack,df_yandexgpt.reset_index()[['question','response']].rename(columns={'response':'yandexgpt'}),left_index=True,right_index=True)[['question','gpt-4o','gpt-3.5-turbo','yandexgpt']].rename(columns={'question':'prompt'})

df_parsing_task = pd.merge(df_responses,df_prompts,how='left',on='prompt').drop(columns=['Unnamed: 0'])

Forming prompts for parsing with GPT-4o.

In [10]:
task = 'There are user query and answers from three llm models. Could you parse these outputs into the following format?\n\n{\n\'model_1\': int, # number from 1 to 5 based on a corresponding model answer, if there are multiple answers then the average of them, or null if it\'s impossible to tell based on answer\n\'model_2\': int, # number from 1 to 5 based on a corresponding model answer, if there are multiple answers then the average of them, or null if it\'s impossible to tell based on answer\n\'model_3\': int, # number from 1 to 5 based on a corresponding model answer, if there are multiple answers then the average of them, or null if it\'s impossible to tell based on answer\n}\nPlease, give your own answer using only format above. Don\'t provide any additional comments. Your data for analysis:\n\n'
df_parsing_task['combined'] = df_parsing_task.apply(lambda row: f"{task}User query: {row['prompt']}\n\nModel 1: {row['gpt-4o']}\n\nModel 2: {row['gpt-3.5-turbo']}\n\nModel 3: {row['yandexgpt']}", axis=1)

Setting up parsing.

In [6]:
import json

with open('api_keys.json') as json_file:
    apikeys = json.load(json_file)

In [7]:
apikey = apikeys['openai']

In [8]:
from openai import OpenAI

def get_response_from_chatgpt(prompt, gptmodel):
    client = OpenAI(api_key=apikey)
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=gptmodel)
    return response.choices[0].message.content

Parsing.

In [ ]:
responses = []

for prompt in df_parsing_task['combined']:
    message = get_response_from_chatgpt(prompt, 'gpt-4o')
    responses.append((prompt,message))

In [6]:
import ast

parsed = []

for (prompt,message) in responses:
    try:        
        parsed.append((prompt,)+tuple(ast.literal_eval(message.replace('null','None')).values()))
    except:
        print(message)
        continue

In [ ]:
df_parsed = pd.DataFrame(parsed, columns=['combined','score_gpt-4o','score_gpt-3.5-turbo','score_yandexgpt'])

df = pd.merge(df_parsing_task,df_parsed,how='left',on='combined')

df.to_csv('responses_parsed.csv', sep=';', encoding='utf-8-sig')
df